Prompt templates help to translate user input and parameters into instructions for a language model. This can be used to guide a model's response, helping it understand the context and generate relevant and coherent language-based output

In [2]:
TEMPLATE = """
You are a helpful assistant that translates the {input_language} to {output_language}
"""

In [3]:
from langchain.prompts.prompt import PromptTemplate

prompt_template = PromptTemplate.from_template(
    template=TEMPLATE
)
prompt_template.format(input_language="english", output_language="german")

'\nYou are a helpful assistant that translates the english to german\n'

In [4]:
prompt_template = PromptTemplate(template=TEMPLATE, input_variables=["input_language", "output_language"])
prompt_template.format(input_language="english", output_language="german")

'\nYou are a helpful assistant that translates the english to german\n'

Few Shot Prompt - provide a few examples in the template

In [5]:
TEMPLATE = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}

Examples:
text: The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.
sentiment: positive
subject: BellaVista

text: BellaVista restaurant was alright. The food was decent, but nothing stood out.
sentiment: neutral
subject: BellaVista

text: I was disappointed with BellaVista. The service was slow and the dishes lacked flavor.
sentiment: negative
subject: BellaVista

text: SeoulSavor offered the most authentic Korean flavors I've tasted outside of Seoul. The kimchi was perfectly fermented and spicy.
sentiment: positive
subject: SeoulSavor

text: SeoulSavor was okay. The bibimbap was good but the bulgogi was a bit too sweet for my taste.
sentiment: neutral
subject: SeoulSavor

text: I didn't enjoy my meal at SeoulSavor. The tteokbokki was too mushy and the service was not attentive.
sentiment: negative
subject: SeoulSavor

text: MunichMeals has the best bratwurst and sauerkraut I've tasted outside of Bavaria. Their beer garden ambiance is truly authentic.
sentiment: positive
subject: MunichMeals

text: MunichMeals was alright. The weisswurst was okay, but I've had better elsewhere.
sentiment: neutral
subject: MunichMeals

text: I was let down by MunichMeals. The potato salad lacked flavor and the staff seemed uninterested.
sentiment: negative
subject: MunichMeals
"""

In [6]:
prompt_template = PromptTemplate(template=TEMPLATE, input_variables=["input"])
prompt_template.format(input="The MunichDeals experience was just awesome!")

"\nInterprete the text and evaluate the text.\nsentiment: is the text in a positive, neutral or negative sentiment?\nsubject: What subject is the text about? Use exactly one word.\n\nFormat the output as JSON with the following keys:\nsentiment\nsubject\n\ntext: The MunichDeals experience was just awesome!\n\nExamples:\ntext: The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.\nsentiment: positive\nsubject: BellaVista\n\ntext: BellaVista restaurant was alright. The food was decent, but nothing stood out.\nsentiment: neutral\nsubject: BellaVista\n\ntext: I was disappointed with BellaVista. The service was slow and the dishes lacked flavor.\nsentiment: negative\nsubject: BellaVista\n\ntext: SeoulSavor offered the most authentic Korean flavors I've tasted outside of Seoul. The kimchi was perfectly fermented and spicy.\nsentiment: positive\nsubject: SeoulSavor\n\ntext: SeoulSavor was okay. The bibimbap was good but the b

LangChain also provides a FewShotPromptTemplate class, which allows creating the examples more modularized

In [7]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
    {
        "text": "The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.",
        "response": "sentiment: positive\nsubject: BellaVista"
    },
    {
        "text": "BellaVista restaurant was alright. The food was decent, but nothing stood out.",
        "response": "sentiment: neutral\nsubject: BellaVista"
    },
    ### other examples are left out
]

In [8]:
new_example = {
    "text": "SeoulSavor was okay. The bibimbap was good but the bulgogi was a bit too sweet for my taste.",
    "response": "sentiment: neutral\nsubject: SeoulSavor"
}
examples.append(new_example)

In [9]:
example_prompt = PromptTemplate(input_variables=["text", "response"], template="Text: {text}\n{response}")

In [10]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="text: {input}",
    input_variables=["input"]
)

In [11]:
print(prompt.format(input="The MunichDeals experience was just awesome!"))

Text: The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.
sentiment: positive
subject: BellaVista

Text: BellaVista restaurant was alright. The food was decent, but nothing stood out.
sentiment: neutral
subject: BellaVista

Text: SeoulSavor was okay. The bibimbap was good but the bulgogi was a bit too sweet for my taste.
sentiment: neutral
subject: SeoulSavor

text: The MunichDeals experience was just awesome!


Composition

You can also compose multiple prompts together. This can be especially useful if you want to reuse parts of prompts.

In [12]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

# Introduction
introduction_template = """
Interprete the text and evaluate the text. Determine if the text has a positive, neutral, or negative sentiment. Also, identify the subject of the text in one word.
"""
introduction_prompt = PromptTemplate.from_template(introduction_template)

# Example
example_template = """
Chain-of-Thought Prompts:
Let's start by evaluating a statement. Consider: "{example_text}". How does this make you feel about {example_subject}?
Response: {example_evaluation}

Based on the {example_sentiment} nature of that statement, how would you format your response?
Response: {example_format}
"""
example_prompt = PromptTemplate.from_template(example_template)

# Execution
execution_template = """
Now, execute this process for the text: "{input}".
"""
execution_prompt = PromptTemplate.from_template(execution_template)

# Composing the full prompt
full_template = """{introduction}

{example}

{execution}"""
full_prompt = PromptTemplate.from_template(full_template)

# PipelinePrompts
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("execution", execution_prompt)
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

In [13]:
print(pipeline_prompt.format(
    example_text="The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.",
    example_subject="BellaVista",
    example_evaluation="It sounds like a positive review for BellaVista.",
    example_sentiment="positive",
    example_format='{ "sentiment": "positive", "subject": "BellaVista" }',
    input="The new restaurant downtown has bland dishes and the wait time is too long."
))


Interprete the text and evaluate the text. Determine if the text has a positive, neutral, or negative sentiment. Also, identify the subject of the text in one word.



Chain-of-Thought Prompts:
Let's start by evaluating a statement. Consider: "The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.". How does this make you feel about BellaVista?
Response: It sounds like a positive review for BellaVista.

Based on the positive nature of that statement, how would you format your response?
Response: { "sentiment": "positive", "subject": "BellaVista" }



Now, execute this process for the text: "The new restaurant downtown has bland dishes and the wait time is too long.".



Serializing prompts
Example is an easy prompt since serializing does not work for the PipelinePromptTemplate (yet)

In [14]:
prompt = PromptTemplate(input_variables=["input"], template="Tell me a joke about {input}")
prompt.save("prompt.yaml")
prompt.save("prompt.json")

In [15]:
from langchain.prompts import load_prompt

prompt = load_prompt("prompt.yaml")
prompt.format(input="chickens")

'Tell me a joke about chickens'

In [16]:
prompt = load_prompt("prompt.json")
prompt.format(input="cows")

'Tell me a joke about cows'

In [4]:
import openai
from google.auth import default, transport
from langchain import PromptTemplate
# Build
from langchain_openai import ChatOpenAI
from vertexai.preview import rag

credentials, _ = default(scopes=['https://www.googleapis.com/auth/cloud-platform'])
auth_request = transport.requests.Request()
credentials.refresh(auth_request)


MODEL_LOCATION = "us-central1"
PROJECT_ID='sacred-alliance-433217-e3'
MODEL_ID = "meta/llama3-405b-instruct-maas"  # @param {type:"string"} ["meta/llama3-405b-instruct-maas"]

client = openai.OpenAI(
    base_url=f"https://{MODEL_LOCATION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{MODEL_LOCATION}/endpoints/openapi/chat/completions?",
    api_key=credentials.token,
)

c:\Users\Chiranjeev R\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [8]:
from openai import llm
from langchain import PromptTemplate
from langchain.chains import LLMChain

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"topic": "funny"}))

ImportError: cannot import name 'llm' from 'openai' (c:\Users\Chiranjeev R\AppData\Local\Programs\Python\Python312\Lib\site-packages\openai\__init__.py)

In [7]:
from client import llm
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
# decouple to read .env variables(OpenAI Key)
# simple sequential chain
from langchain.chains import SimpleSequentialChain
# memory in sequential chain
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory
from datetime import datetime



# rapper
rapper_template: str = """You are an American rapper, your job is to come up with\
lyrics based on a given topic

Here is the topic you have been asked to generate a lyrics on:
{input}\
"""

rapper_prompt_template: PromptTemplate = PromptTemplate(
    input_variables=["input"], template=rapper_template)

# creating the rapper chain
rapper_chain: LLMChain = LLMChain(
    llm=llm, output_key="lyric", prompt=rapper_prompt_template)

# verifier
verifier_template: str = """You are a verifier of rap songs, you are tasked\
to inspect the lyrics of rap songs. If they consist of violence and abusive languge\
flag the lyrics. 

Your response should be in the following format, as a Python Dictionary.
lyric: this should be the lyric you received 
Violence_words: True or False

Here is the lyrics submitted to you:
{lyric}\
"""

verified_prompt_template: PromptTemplate = PromptTemplate(
    input_variables=["lyric"], template=verifier_template)

# creating the verifier chain
verifier_chain: LLMChain = LLMChain(
    llm=llm, output_key="AI_verified", prompt=verified_prompt_template)


# final output chain
final_template: str = """You are a final quality assurance of a lyrics post.\
Your job will be to accept a lyric and output data in the following format

Your final response should be in the following format, in a Python Dictionary format:
lyric: this should be the lyric you received
Date and time verified: {time_created_and_verified}
Verified by human: {verified_by_human}

Here is the lyric submitted to you:
{AI_verified}\
"""

final_prompt_template: PromptTemplate = PromptTemplate(
    input_variables=["AI_verified", "time_created_and_verified", "verified_by_human"], template=final_template)

# creating the verifier chain
final_chain: LLMChain = LLMChain(
    llm=llm, output_key="final_output", prompt=final_prompt_template)


# creating the simple sequential chain
ss_chain: SequentialChain = SequentialChain(
    memory=SimpleMemory(memories={
                        "time_created_and_verified": str(datetime.utcnow()), "verified_by_human": "False"}),
    chains=[rapper_chain, verifier_chain, final_chain],
    # multiple variables
    input_variables=["input"],
    output_variables=["final_output"],
    verbose=True)

# running chain
review = ss_chain.run(input="christ worship songs")
print(review)

NameError: name 'llm' is not defined

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI
from client import llm


joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | llm
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | llm
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

response=map_chain.invoke({"topic": "bear"})

print(response)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI
from client import llm
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate


chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | llm
    | StrOutputParser()
)

reponse=chain.invoke({"question": "how do I call Anthropic?"})
print(reponse)

langchain_chain = PromptTemplate.from_template(
    """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | llm
anthropic_chain = PromptTemplate.from_template(
    """You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | llm
general_chain = PromptTemplate.from_template(
    """Respond to the following question:

Question: {question}
Answer:"""
) | llm

def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
) | StrOutputParser()

final_reponse=full_chain.invoke({"question": "how do I use Anthropic?"})
print(final_reponse)

In [ ]:
from client import llm
from langchain import PromptTemplate
from langchain.chains import LLMChain

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")

chain = prompt | llm | StrOutputParser()

analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")

composed_chain = {"joke": chain} | analysis_prompt | llm | StrOutputParser()

reponse=composed_chain.invoke({"topic": "bears"})
print(reponse)

In [12]:
import openai
from google.auth import default, transport
from langchain import PromptTemplate
# Build
from langchain_openai import ChatOpenAI
from vertexai.preview import rag

credentials, _ = default(scopes=['https://www.googleapis.com/auth/cloud-platform'])
auth_request = transport.requests.Request()
credentials.refresh(auth_request)


MODEL_LOCATION = "us-central1"
PROJECT_ID='sacred-alliance-433217-e3'
MODEL_ID = "meta/llama3-405b-instruct-maas"  # @param {type:"string"} ["meta/llama3-405b-instruct-maas"]

client = openai.OpenAI(
    base_url=f"https://{MODEL_LOCATION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{MODEL_LOCATION}/endpoints/openapi/chat/completions?",
    api_key=credentials.token,
)

c:\Users\Chiranjeev R\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [14]:

from langchain import PromptTemplate
from langchain.chains import LLMChain

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "write the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:"),
         ("human","{equation_statement}"),
         ]
)

model = ChatOpenAI(temperature=0)

runnable = (
    {"equation_statement":runnablepassthrough()}
    | prompt 
    | model.bind(stop="SOLUTION")
    |StrOutputParser()
    )
print(runnable.invoke('x raised to the third plus seven equals 12'))

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

def length_function(text):
    return len(text)

def _multiple_length_function(text):
    return len(text1) * len(text2)

def multiple_length_function(_dict): 
    return _multiple_length_function(_dict["text1"],_dict["text2"])

model = ChatOpenAI

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain1 = prompt|model 

chain = (
    {"a":itemgetter("foo") | RunnableLambda(length_function),
     "b":{"text1":itemgetter("foo"),"text2":itemgetter("bar")}
     | RunnableLambda(multiple_length_function),
     }
     |prompt | model

)

chain.invoke({"foo":"bar" , "bar":"gra"})